<a href="https://colab.research.google.com/github/hanah512/deep-learning/blob/master/PyTorchTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 60-minute Blitz


In [1]:
import torch

In [6]:
sizes = 4,6,2
x = torch.tensor(sizes)
print(x)

tensor([4, 6, 2])
